In [3]:
import glob
#paths = glob.glob('./texts/*/*.txt')
paths_perseus = glob.glob('./texts/perseus/*.txt')
paths_proiel = glob.glob('./texts/proiel-txt/*.txt')
paths_harrington = glob.glob('./texts/harrington-txt/*.txt')

word_list_ph = []
word_list_proiel = []

In [7]:
def get_sentences(paths, word_list, sentence_delimiter):
    for p in paths:
        #sample_text_path = "./texts/perseus/phi0448.phi001.perseus-lat1.tb.txt"
        file = open(p)
        string = file.read()
        text = string
        file.close()
        for i in range(10):
            s = '[' + str(i) + ']/' 
            text = text.replace(s,'')
        text = text.replace('?', '.')
        text = text.replace('!', '.')
        if sentence_delimiter != '\n':
            text = text.replace('\n','')
        text = text.replace('[0]/','')
        text = text.replace('[1]/','')
        sentence_list = text.split(sentence_delimiter)
        #print(len(sentence_list))
        strip = {'',',',';'}
        for sent in sentence_list:
            words = sent.split(' ')
            tuple_list = []
            for word in words:
                word_tag_pair = word.split('/')
                if (len(word_tag_pair) == 2) and (word_tag_pair[0] not in strip):
                    tuple_list.append( (word_tag_pair[0], word_tag_pair[1]) )
            if len(tuple_list) > 0:
                word_list.append(tuple_list)                

In [8]:
get_sentences(paths_perseus, word_list_ph,  '.')
get_sentences(paths_harrington, word_list_ph, '.')
get_sentences(paths_proiel, word_list_proiel, '\n')

In [10]:
print(len(word_list_ph))
print(len(word_list_proiel))

7524
19425


In [12]:
import pickle
with open('ph_corpus', 'wb') as ph:
    pickle.dump(word_list_ph, ph)

with open('proiel_corpus', 'wb') as proiel:
    pickle.dump(word_list_proiel, proiel)

In [6]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

In [7]:
def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

In [8]:
train_size = int(.75*len(word_list))
train_sent = word_list[:train_size]
test_sent = word_list[train_size:]
def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            #print(tagged)
            X.append(features(untag(tagged), index))
            y.append(tagged[index][1])
 
    return X, y
 
X, y = transform_to_dataset(train_sent)
print(len(X))

266808


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
 
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy'))
])
 
clf.fit(X[:50000], y[:50000])   # Use only the first 10K samples if you're running it multiple times. It takes a fair bit :)
 
print ('Training completed')
 
X_test, y_test = transform_to_dataset(test_sent)
 
print ("Accuracy:" + str(clf.score(X_test, y_test)))